In [2]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import ast
from pandas import json_normalize
import os

ETL - Steam_games

In [17]:
df = pd.read_csv("../01 - Extracción de archivos/Juegos_Steam.csv")

In [88]:
Juegos = df[["id", "title", "genres", "release_date", "specs", "price"]]
Juegos = Juegos.rename(columns={'id': 'Id_juego', "title": "Nombre_juego", "genres": "Genero", "release_date": "Año_Lanzamiento","specs" : "Formato", "price" : "Pago" })
Juegos = Juegos.dropna()
Juegos['Id_juego'] = Juegos['Id_juego'].round().astype(int)  ##convierto los datos en tipo entero
Juegos["Genero"]= Juegos["Genero"].apply(literal_eval)  ##convierto la cadena en estructura de datos

In [89]:
## Como en la columna "Año_Lanzamiento" tengo cadena tanto en formato fecha corta como de tipo str como puede 
## ser "June 2009", lo que propongo es una función que extraiga todos aquellas cadenas que sean de tipo "YYYY"

def extraer_años(texto):
    # coincidencias con el formato "Año XXXX"
    coincidencias = re.findall(r'\b\d{4}\b', texto)

    # Devuelve la lista de coincidencias encontradas o None si no se encuentran años
    return int(coincidencias[0]) if coincidencias else None

In [90]:
Juegos["Año_Lanzamiento"] = Juegos["Año_Lanzamiento"].apply(extraer_años)
Juegos = Juegos.dropna(subset=["Año_Lanzamiento"] )
Juegos['Año_Lanzamiento'] = Juegos['Año_Lanzamiento'].round().astype(int)
Juegos["Formato"]= Juegos["Formato"].apply(literal_eval)

In [91]:
## En la columna "Pago" convierto los valores decimales en 1 referente a juego pago y en 0 referente a juego gratis

def convertir_valor(valor):
    try:
        # Intenta convertir el valor a decimal
        float_valor = float(valor)
        return 1
    except ValueError:
        # Si la conversión a decimal falla, trata el valor como cadena
        return 0

In [92]:
Juegos["Pago"] = Juegos["Pago"].apply(convertir_valor)
Juegos

,Id_juego,Nombre_juego,Genero,Año_Lanzamiento,Formato,Pago
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018,[Single-player],1
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018,"[Single-player, Multi-player, Online Multi-Pla...",0
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017,"[Single-player, Multi-player, Online Multi-Pla...",0
3,767400,弹炸人2222,"[Action, Adventure, Casual]",2017,[Single-player],1
5,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018,"[Single-player, Steam Achievements]",1
...,...,...,...,...,...,...
32129,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018,"[Single-player, Steam Achievements, Steam Cloud]",1
32130,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018,"[Single-player, Steam Achievements]",1
32131,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018,"[Single-player, Steam Achievements, Steam Clou...",1
32132,610660,Russian Roads,"[Indie, Racing, Simulation]",2018,"[Single-player, Steam Achievements, Steam Trad...",1


In [94]:
Juegos.to_csv("../02 - Desanidado de datos y limpieza/Juegos_ETL.csv", index=True)

ETL - User_reviews

In [134]:
df1 = pd.read_csv("../01 - Extracción de archivos/Reviews_users.csv")
Reviews = df1.drop(columns={"user_url"}, axis=1)
Reviews.head(2)

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


In [135]:
## Función para convertir la cadena en una lista de diccionarios usando ast.literal_eval de manera segura
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else []
    except (SyntaxError, ValueError):
        return []

In [136]:
# Aplicar la función a cada fila de 'nueva_columna'
Reviews['reviews'] = Reviews['reviews'].apply(safe_literal_eval)
Reviews['reviews'].iloc[0][0]

{'funny': '',
 'posted': 'Posted November 5, 2011.',
 'last_edited': '',
 'item_id': '1250',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}

In [137]:
## La columna "reviews" esta compuesta por listas que contiene a su vez varios diccionarios.
## Como ya conozco las claves de los mismos, creo funciones que las utilicen para extraer los valores.

# Funciones para extraer valores de las claves en cada diccionario
def extraer_item_id(lista_diccionarios):
    return [diccionario.get('item_id', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

def extraer_item_name(lista_diccionarios):
    return [diccionario.get('recommend', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

def extraer_playtime_forever(lista_diccionarios):
    return [diccionario.get('review', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

def extraer_posted(lista_diccionarios):
    return [diccionario.get('posted', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

# Crear un nuevo DataFrame con los resultados
df_resultado = pd.DataFrame()

# Crear nuevas columnas para cada clave en el nuevo DataFrame
df_resultado['item_id'] = Reviews['reviews'].apply(extraer_item_id)
df_resultado['recommend'] = Reviews['reviews'].apply(extraer_item_name)
df_resultado['review'] = Reviews['reviews'].apply(extraer_playtime_forever)
df_resultado['posted'] = Reviews['reviews'].apply(extraer_posted)

In [138]:
Reviews = pd.concat([Reviews, df_resultado], axis=1)
Reviews = Reviews.drop(columns={'reviews'}, axis=1)
Reviews = Reviews.explode(['item_id', 'recommend', 'review', 'posted'])
Reviews

,user_id,item_id,recommend,review,posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,"Posted November 5, 2011."
0,76561197970982479,22200,True,It's unique and worth a playthrough.,"Posted July 15, 2011."
0,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,"Posted April 21, 2011."
1,js41637,251610,True,I know what you think when you see this title ...,"Posted June 24, 2014."
1,js41637,227300,True,For a simple (it's actually not all that simpl...,"Posted September 8, 2013."
...,...,...,...,...,...
25797,76561198312638244,70,True,a must have classic from steam definitely wort...,Posted July 10.
25797,76561198312638244,362890,True,this game is a perfect remake of the original ...,Posted July 8.
25798,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,Posted July 3.
25798,LydiaMorley,730,True,:D,Posted July 20.


In [139]:
# Definir una función para extraer el año utilizando expresiones regulares
def extraer_anio(fecha):
    if pd.notna(fecha) and isinstance(fecha, str):
        match = re.search(r'\b\d{4}\b', fecha)
        return int(match.group()) if match else None
    return None

In [140]:
# Aplicar la función para extraer el año y crear una nueva columna 'anio'
Reviews['posted'] = Reviews['posted'].apply(extraer_anio)
# Eliminar filas con valores NaN en la columna 'anio'
Reviews = Reviews.dropna(subset=['posted'])
# Convertir la columna 'anio' a tipo entero
Reviews['posted'] = Reviews['posted'].astype(int)
Reviews

C:\Users\atenc\AppData\Local\Temp\ipykernel_7688\4268006109.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Reviews['posted'] = Reviews['posted'].astype(int)


,user_id,item_id,recommend,review,posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011
0,76561197970982479,22200,True,It's unique and worth a playthrough.,2011
0,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
1,js41637,251610,True,I know what you think when you see this title ...,2014
1,js41637,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
25764,wayfeng,730,True,its FUNNNNNNNN,2015
25765,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015
25769,72947282842,730,True,Prettyy Mad Game,2015
25771,ApxLGhost,730,True,AMAZING GAME 10/10,2015


In [141]:
# Descargar el lexicon VADER para el análisis de sentimiento
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\atenc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [142]:
# Llenar los valores NaN con una cadena vacía
Reviews['review'] = Reviews['review'].fillna('')
# Inicializar el analizador de sentimientos de VADER
sia = SentimentIntensityAnalyzer()
# Aplicar análisis de sentimientos y asignar valores a una nueva columna
Reviews['sentiment_analysis'] = Reviews['review'].apply(lambda x: 2 if sia.polarity_scores(x)['compound'] >= 0.05 else (1 if -0.05 < sia.polarity_scores(x)['compound'] < 0.05 else 0))

Reviews = Reviews.drop(columns={'review'}, axis=1)
Reviews

C:\Users\atenc\AppData\Local\Temp\ipykernel_7688\2494774130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Reviews['review'] = Reviews['review'].fillna('')
C:\Users\atenc\AppData\Local\Temp\ipykernel_7688\2494774130.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Reviews['sentiment_analysis'] = Reviews['review'].apply(lambda x: 2 if sia.polarity_scores(x)['compound'] >= 0.05 else (1 if -0.05 < sia.polarity_scores(x)['compound'] < 0.05 else 0))


,user_id,item_id,recommend,posted,sentiment_analysis
0,76561197970982479,1250,True,2011,2
0,76561197970982479,22200,True,2011,2
0,76561197970982479,43110,True,2011,2
1,js41637,251610,True,2014,2
1,js41637,227300,True,2013,2
...,...,...,...,...,...
25764,wayfeng,730,True,2015,1
25765,76561198251004808,253980,True,2015,2
25769,72947282842,730,True,2015,0
25771,ApxLGhost,730,True,2015,2


In [143]:
Reviews.to_csv("../02 - Desanidado de datos y limpieza/Reviews_ETL.csv", index=True)

ETL - User_items

In [144]:
df2 = pd.read_csv("../01 - Extracción de archivos/Items_users.csv")

In [145]:
df2.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [146]:
Items = df2.drop(columns={"items_count", "steam_id", "user_url" }, axis=1)

# Función para convertir la cadena en una lista de diccionarios usando ast.literal_eval de manera segura
Items['items'] = Items['items'].apply(safe_literal_eval)
Items

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...
88305,76561198323066619,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,[]
88308,76561198329548331,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [147]:
# Funciones para extraer valores de las claves en cada diccionario
def extraer_item_id(lista_diccionarios):
    return [diccionario.get('item_id', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

def extraer_item_name(lista_diccionarios):
    return [diccionario.get('item_name', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

def extraer_playtime_forever(lista_diccionarios):
    return [diccionario.get('playtime_forever', None) for diccionario in lista_diccionarios] if isinstance(lista_diccionarios, list) else []

# Crear un nuevo DataFrame con los resultados
df_resultado = pd.DataFrame()

# Crear nuevas columnas para cada clave en el nuevo DataFrame
df_resultado['item_id'] = Items['items'].apply(extraer_item_id)
df_resultado['item_name'] = Items['items'].apply(extraer_item_name)
df_resultado['playtime_forever'] = Items['items'].apply(extraer_playtime_forever)

In [149]:
# Concatenar df_resultado con Items
Items = pd.concat([Items, df_resultado], axis=1)
Items = Items.drop(columns={'items'}, axis=1)

In [150]:
Items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ..."
1,js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507..."
3,Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,..."
4,doctr,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47..."
...,...,...,...,...
88305,76561198323066619,"[413850, 413851, 413852, 413853, 413854, 41385...","[CS:GO Player Profiles, CS:GO Player Profiles:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88306,76561198326700687,"[11020, 6370, 13140, 51100, 107900, 113400, 20...","[TrackMania Nations Forever, Bloodline Champio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88307,XxLaughingJackClown77xX,[],[],[]
88308,76561198329548331,"[304930, 227940, 346330, 373330, 388490, 52157...","[Unturned, Heroes & Generals, BrainBread 2, Al...","[677, 43, 0, 0, 3, 4, 3]"


In [151]:
# Utilizar explode para descomponer solo las listas 'item_id', 'item_name', y 'playtime_forever'
Items = Items.explode(['item_id', 'item_name', 'playtime_forever'])
# Agrupar por 'user_id', 'item_id', y 'item_name' y sumar 'playtime_forever'
Items = Items.groupby(['user_id', 'item_id', 'item_name'], as_index=False)['playtime_forever'].sum()
Items

,user_id,item_id,item_name,playtime_forever
0,--000--,104700,Super Monday Night Combat,461
1,--000--,104900,ORION: Prelude,63
2,--000--,105600,Terraria,1172
3,--000--,113400,APB Reloaded,2763
4,--000--,1250,Killing Floor,2949
...,...,...,...,...
5094077,zzzmidmiss,94600,Hector: Ep 1,0
5094078,zzzmidmiss,94610,Hector: Ep 2,0
5094079,zzzmidmiss,94620,Hector: Ep 3,0
5094080,zzzmidmiss,9480,Saints Row 2,7


In [152]:
Items.to_csv("../02 - Desanidado de datos y limpieza/Items_ETL.csv", index=True)

In [ ]:
##eliminar los archivos CSV extraidos (archivos muy pesados los cuales no voy a dar uso)
os.remove("../01 - Extracción de archivos/Items_users.csv")

In [ ]:
os.remove("../01 - Extracción de archivos/Juegos_Steam.csv")

In [5]:
os.remove("../01 - Extracción de archivos/Reviews_users.csv")